In [1]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import ipywidgets as widgets
from ipywidgets import interact
from IPython.display import display, clear_output

def get_category_colour(category):
    
    if category == "normal":
        return "#17becf"
    elif category == "glucose":
        return "#2ca02c"
    elif category == "athlete":
        return "#9467bd"
    elif category == "protein":
        return "#e377c2"
    
    
def build_size_list_and_hover_text_per_category(category,barcode,scores):
    
    sub_scores = scores[scores.category == category]
    found_sample_index_match = sub_scores["sample_id"].str.contains("^"+barcode+"$").values
    size_list = []
    hover_text = []
    default_point_size = 10
    selected_point_size = 20
    ns,nc = sub_scores.shape
    i = 0
    while (i < ns):
        if found_sample_index_match[i] == True:
            size_list.append(selected_point_size)
        else:
            size_list.append(default_point_size)
        hover_text.append("Sample: "+sub_scores.iloc[i].sample_id) 
        i = i + 1
    return size_list,hover_text

def plot_scores_per_category(fig,category,barcode,scores):
    
    size_list, hover_text = build_size_list_and_hover_text_per_category(category,barcode,scores)
    fig.add_trace(go.Scatter(
                        x=scores[scores.category == category].pc1.values,
                        y=scores[scores.category == category].pc2.values,
                        name=category,
                        mode='markers',
                        marker=dict(size=size_list,color=get_category_colour(category)),
                        hovertext=hover_text,
                        hoverinfo="all",
                   ))
    
    return fig

fig_scores = go.FigureWidget()
fig_spectra = go.FigureWidget()

def plot_scores(barcode,scores):
 
 #   fig = go.FigureWidget()
    fig_scores = plot_scores_per_category(fig_scores,"normal",barcode,scores)
    fig_scores = plot_scores_per_category(fig_scores,"glucose",barcode,scores)
    fig_scores = plot_scores_per_category(fig_scores,"athlete",barcode,scores)
    fig_scores = plot_scores_per_category(fig_scores,"protein",barcode,scores)
    fig_scores.update_layout(title="PCA scores plot of PC1 and PC2",
                       xaxis_title="PC1",
                       yaxis_title="PC2",
                       showlegend=True)
    fig_scores.update_xaxes(range=[-1, 1])
    fig_scores.update_yaxes(range=[-1, 1])
#    fig.show()
    
    return fig_scores
    
def plot_spectra(barcode,scores,spectra):
    
    category = scores[scores.sample_id == barcode].category.values[0]
    #fig = go.FigureWidget()
    fig_spectra.add_trace(go.Scatter(
                        x=spectra.index.values,
                        y=spectra[barcode],
                        name=barcode,
                        mode='lines',
                        line=dict(color=get_category_colour(category))
                     ))
    fig_spectra.add_trace(go.Scatter(
                    x=spectra.index.values,
                    y=spectra['Spectra/310'],
                    name="normal",
                    mode='lines',
                    line=dict(color=get_category_colour('normal'))
                 ))
    fig_spectra.update_xaxes(range=[10,-0.2])
    fig_spectra.update_layout(title="NMR Spectrum for sample "+barcode,
                      xaxis_title="ppm",
                       yaxis_title="Intensity")
#    fig.show()
    return fig_spectra
    

def check_sample(barcode,scores):
    
    # If the sample id is not found:
    if len(scores[scores.sample_id == barcode].values) == 0:
        print("Sample " + barcode + " not found")
        
        
#barcode_widget.observe(on_value_change, names='value')
#def on_change(change):
#    if change['type'] == 'change' and change['name'] == 'value':
#        #identify_sample(change['new'],scores,spectra)        
#        pass
    
        
spectra = pd.read_csv("spectra.csv").T
new_header = spectra.iloc[0] #grab the first row for the header
spectra = spectra[1:] #take the data less the header row
spectra.columns = new_header
scores = pd.read_csv('scores.csv')

barcode = "Spectra/310"

barcode_widget = widgets.Text(
    value='',
    placeholder='scan barcode',
    description='Barcode:',
    disabled=False
)

check_sample(barcode,scores)

@interact(barcode="Spectra/310")
def update(barcode):
    with fig_spectra.batch_update():
        fig_scores = plot_scores(barcode,scores)
        fig_scores.show()
        fig_spectra = plot_spectra(barcode,scores,spectra)
        fig_spectra.show()

        
        #fig_scores.delete_trace(name='Spectra/310')
        

interactive(children=(Text(value='Spectra/310', description='barcode'), Output()), _dom_classes=('widget-inter…

In [2]:

barcode_widget.observe(on_change)        
display(barcode_widget)

#Spectra/310:Normal
#Spectra/130:Athlete
#Spectra/840:Glucose
#Spectra/640:Protein

NameError: name 'on_change' is not defined